# Урок 7. Модель Transformer.

Поэкспериментировать с моделью генерации текста, попробовать модель на словах, посмотреть что отработает лучше<br>
Поэкспериментировать с переводом написать по вашим наблюдениям где он ошибается, попробовать изменить архитектуру

#### 1. Поэкспериментировать с моделью генерации текста, попробовать модель на словах, посмотреть что отработает лучше

#### Генерация на буквах

In [1]:
import tensorflow as tf

import numpy as np
import os
import time
import re

In [2]:
files = [
    'Blok_A._Spisokshkolnoy._Polnoe_Sobranie_Stihotvoreniyi.txt',
    'Cvetaeva_M._Polnoe_Sobranie_Stihotvoreniyi.txt',
    'Esenin_S._Spisokshkolnoy._Polnoe_Sobranie_Stihotvoreniyi.txt',
    'Lermontov_M._Spisokshkolnoy._Mcyiri.txt',
    'Nekrasov_N._Stihotvoreniya.txt',
    'Pushkin_A._Mednyiyi_Vsadnik.txt',
    'Pushkin_A._Ruslan_I_Lyudmila.txt',
    'Pushkin_A._Spisokshkolnoy._Evgeniyi_Onegin.txt',
    'Pushkin_A._Spisokshkolnoy._Polnoe_Sobranie_Stihotvoreniyi.txt'
]

In [3]:
text = ''

for f in files:
    text = text + '\n' + open('txt/' + f, 'rt').read()

In [4]:
len(text)

2872846

In [5]:
print(text[:500])
text[:500]


Александр Александрович Блок

Полное собрание стихотворений





Стихотворения 1898 года





«Пусть светит месяц – ночь темна…»


		Пусть светит месяц – ночь темна.
		Пусть жизнь приносит людям счастье, —
		В моей душе любви весна
		Не сменит бурного ненастья.
		Ночь распростерлась надо мной
		И отвечает мертвым взглядом
		На тусклый взор души больной,
		Облитой острым, сладким ядом.
		И тщетно, страсти затая,
		В холодной мгле передрассветной
		Среди толпы блуждаю я
		С одной лишь думою завет


'\nАлександр Александрович Блок\n\nПолное собрание стихотворений\n\n\n\n\n\nСтихотворения 1898 года\n\n\n\n\n\n«Пусть светит месяц – ночь темна…»\n\n\n\t\tПусть светит месяц – ночь темна.\n\t\tПусть жизнь приносит людям счастье, —\n\t\tВ моей душе любви весна\n\t\tНе сменит бурного ненастья.\n\t\tНочь распростерлась надо мной\n\t\tИ отвечает мертвым взглядом\n\t\tНа тусклый взор души больной,\n\t\tОблитой острым, сладким ядом.\n\t\tИ тщетно, страсти затая,\n\t\tВ холодной мгле передрассветной\n\t\tСреди толпы блуждаю я\n\t\tС одной лишь думою завет'

In [6]:
text_re = re.sub(r'\xa0', ' ', text)
text_re = re.sub(r'\t', '', text_re)
text_re = re.sub(r'[^а-яА-Я\n ]+', '', text_re)
text_re = re.sub(r'[ ]{2,}', ' ', text_re)
text_re = re.sub(r' \n', '\n', text_re)
text_re = re.sub(r'\n ', '\n', text_re)
text_re = re.sub(r'[\n]{3,}', '\n\n', text_re)
text_re = re.sub(r'[\n]{2,}[а-яА-Я ]+[\n]{2,}', '\n\n', text_re)
text_re = re.sub(r'[\n]{2,}[а-яА-Я ]+[\n]{2,}', '\n\n', text_re)
text_re = re.sub(r'(^[\n])[а-яА-Я ]+[\n]{2,}', '', text_re)
text = text_re

In [7]:
pos = 0
print(text[pos:pos+500])

Пусть светит месяц ночь темна
Пусть жизнь приносит людям счастье
В моей душе любви весна
Не сменит бурного ненастья
Ночь распростерлась надо мной
И отвечает мертвым взглядом
На тусклый взор души больной
Облитой острым сладким ядом
И тщетно страсти затая
В холодной мгле передрассветной
Среди толпы блуждаю я
С одной лишь думою заветной
Пусть светит месяц ночь темна
Пусть жизнь приносит людям счастье
В моей душе любви весна
Не сменит бурного ненастья

Одной тебе тебе одной
Любви и счастия царице
Те


In [8]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

64 unique characters


In [9]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
# The maximum length sentence you want for a single input in characters
seq_length = 50 #100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

П
у
с
т
ь


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(4):
    print(repr(''.join(idx2char[item.numpy()])))

'Пусть светит месяц ночь темна\nПусть жизнь приносит '
'людям счастье\nВ моей душе любви весна\nНе сменит бур'
'ного ненастья\nНочь распростерлась надо мной\nИ отвеч'
'ает мертвым взглядом\nНа тусклый взор души больной\nО'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in  dataset.take(2):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Пусть светит месяц ночь темна\nПусть жизнь приносит'
Target data: 'усть светит месяц ночь темна\nПусть жизнь приносит '
Input data:  'людям счастье\nВ моей душе любви весна\nНе сменит бу'
Target data: 'юдям счастье\nВ моей душе любви весна\nНе сменит бур'


In [14]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 50), (128, 50)), types: (tf.int32, tf.int32)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 1024

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 50, 64) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 512)          32768     
_________________________________________________________________
gru (GRU)                    (128, None, 1024)         4724736   
_________________________________________________________________
gru_1 (GRU)                  (128, None, 1024)         6297600   
_________________________________________________________________
gru_2 (GRU)                  (128, None, 1024)         6297600   
_________________________________________________________________
dense (Dense)                (128, None, 64)           65600     
Total params: 17,418,304
Trainable params: 17,418,304
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
print("Input: \n", repr("".join(idx2char[[input_example_batch[0]]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ны\nНа землю сумерки сходили\nИ вечности вставали сн'

Next Char Predictions: 
 'фюпфЯвШИпнюЖЧЭчМпЯяАфжКкСЦаьЩюЩжАЭХБлФмСзьдЩХциВйэ'


C:\Users\rseme\.conda\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [22]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 50, 64)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.157918


In [23]:
model.compile(optimizer='adam', loss=loss)

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=10,
    save_weights_only=True)

In [25]:
EPOCHS = 20

In [26]:
%%time
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
299/299 [==============================] - 30s 88ms/step - loss: 2.8211
Epoch 2/20
299/299 [==============================] - 27s 87ms/step - loss: 1.8721
Epoch 3/20
299/299 [==============================] - 26s 86ms/step - loss: 1.7183
Epoch 4/20
299/299 [==============================] - 26s 86ms/step - loss: 1.6479
Epoch 5/20
299/299 [==============================] - 27s 87ms/step - loss: 1.5970
Epoch 6/20
299/299 [==============================] - 26s 87ms/step - loss: 1.5502
Epoch 7/20
299/299 [==============================] - 26s 87ms/step - loss: 1.5082
Epoch 8/20
299/299 [==============================] - 26s 87ms/step - loss: 1.4690
Epoch 9/20
299/299 [==============================] - 26s 87ms/step - loss: 1.4269
Epoch 10/20
299/299 [==============================] - 27s 87ms/step - loss: 1.3867
Epoch 11/20
299/299 [==============================] - 27s 87ms/step - loss: 1.3479
Epoch 12/20
299/299 [==============================] - 27s 89ms/step - loss: 1.3102
E

In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_20'

In [28]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            32768     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           4724736   
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
gru_5 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_1 (Dense)              (1, None, 64)             65600     
Total params: 17,418,304
Trainable params: 17,418,304
Non-trainable params: 0
_________________________________________________________________


In [30]:
def generate_text(model, start_string, temperature, num_generate):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    #num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    #temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [40]:
print(generate_text(model, start_string=u"Златая цепь на дубе ", temperature=0.5, num_generate=300))

Златая цепь на дубе полной
Благословлять ее не могла
Довольно воспал в тревоге
Непроницала

И сердцем молод лавр и кипит
От всех тревогой и прочь
Что ж на сердце имена Свидетели мы теперь и молвил
Посмотри желание мощи
И не сойдет ли с досады гордый лорнет
В степь и венца нет мочи
И пленниц Петрова приятель
Порох в три


#### Геренерация на словах

In [66]:
text_tok = re.split(r'(\s+)', text)
text_tok[:20]

['Пусть',
 ' ',
 'светит',
 ' ',
 'месяц',
 ' ',
 'ночь',
 ' ',
 'темна',
 '\n',
 'Пусть',
 ' ',
 'жизнь',
 ' ',
 'приносит',
 ' ',
 'людям',
 ' ',
 'счастье',
 '\n']

In [67]:
vocab = sorted(set(text_tok))
print('{} unique characters'.format(len(vocab)))

67871 unique characters


In [68]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text_tok])

In [69]:
# The maximum length sentence you want for a single input in characters
seq_length = 10
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

Пусть
 
светит
 
месяц


In [70]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(3):
    print(repr(''.join(idx2char[item.numpy()])))

'Пусть светит месяц ночь темна\nПусть'
' жизнь приносит людям счастье\nВ '
'моей душе любви весна\nНе сменит'


In [71]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [72]:
for input_example, target_example in dataset.take(2):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Пусть светит месяц ночь темна\n'
Target data: ' светит месяц ночь темна\nПусть'
Input data:  ' жизнь приносит людям счастье\nВ'
Target data: 'жизнь приносит людям счастье\nВ '


In [73]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 10), (128, 10)), types: (tf.int32, tf.int32)>

In [74]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 512

# Number of RNN units
rnn_units = 1024

In [75]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [76]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 10, 67871) # (batch_size, sequence_length, vocab_size)


In [77]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (128, None, 512)          34749952  
_________________________________________________________________
gru_9 (GRU)                  (128, None, 1024)         4724736   
_________________________________________________________________
gru_10 (GRU)                 (128, None, 1024)         6297600   
_________________________________________________________________
gru_11 (GRU)                 (128, None, 1024)         6297600   
_________________________________________________________________
dense_3 (Dense)              (128, None, 67871)        69567775  
Total params: 121,637,663
Trainable params: 121,637,663
Non-trainable params: 0
_________________________________________________________________


In [78]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [79]:
print("Input: \n", repr("".join(idx2char[[input_example_batch[0]]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'проклятых числ\n\nМой монастырь где '

Next Char Predictions: 
 'ПрободающихпутиноюсахаркаковаРешусьобведеныДнитатариномдобродетелипарнасский'


C:\Users\rseme\.conda\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [80]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 10, 67871)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       11.125151


In [81]:
model.compile(optimizer='adam', loss=loss)

In [82]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=5,
    save_weights_only=True)

In [83]:
EPOCHS = 5

In [84]:
%%time
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
465/465 [==============================] - 128s 270ms/step - loss: 6.6247
Epoch 2/5
465/465 [==============================] - 125s 268ms/step - loss: 6.2726
Epoch 3/5
465/465 [==============================] - 126s 270ms/step - loss: 6.3533
Epoch 4/5
465/465 [==============================] - 128s 274ms/step - loss: 6.3658
Epoch 5/5
465/465 [==============================] - 127s 274ms/step - loss: 6.3688
Wall time: 10min 37s


In [85]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_5'

In [86]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [87]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 512)            34749952  
_________________________________________________________________
gru_12 (GRU)                 (1, None, 1024)           4724736   
_________________________________________________________________
gru_13 (GRU)                 (1, None, 1024)           6297600   
_________________________________________________________________
gru_14 (GRU)                 (1, None, 1024)           6297600   
_________________________________________________________________
dense_4 (Dense)              (1, None, 67871)          69567775  
Total params: 121,637,663
Trainable params: 121,637,663
Non-trainable params: 0
_________________________________________________________________


In [90]:
def generate_text(model, start_string, temperature, num_generate):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    #num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    #temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (''.join(start_string) + ''.join(text_generated))

In [93]:
print(generate_text(model, start_string=['Златая', ' ', 'цепь', ' ', 'на', ' ', 'дубе'], temperature=1, num_generate=50))

Златая цепь на дубе   
 
   ночейКипя с
  и 
искрой Конечно наНе ОндушоюпустынеИ
  твоего    наша здесь переступившийчасовой низвергнутся ДухуяЧто


##### Выводы:
На словах обучить модель не получилось. Обучение застревает на loss ~ 6. Тогда как на буквах обучение проходит довольно гладко и loss ~ 1. Поэтому на буквах отработало лучше, но до идеала далековато.

#### 2. Поэкспериментировать с переводом написать по вашим наблюдениям где он ошибается, попробовать изменить архитектуру

In [85]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [86]:
path_to_file = "./content/rus.txt"

In [87]:
def preprocess_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

    w = w.strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [88]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

In [89]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [90]:
en, ru = create_dataset(path_to_file, None)
print(en[-1])
print(ru[-1])

<start> doubtless there exists in this world precisely the right woman for any given man to marry and vice versa but when you consider that a human being has the opportunity of being acquainted with only a few hundred people , and out of the few hundred that there are but a dozen or less whom he knows intimately , and out of the dozen , one or two friends at most , it will easily be seen , when we remember the number of millions who inhabit this world , that probably , since the earth was created , the right man has never yet met the right woman . <end>
<start> несомненно , для каждого мужчины в этом мире где то есть подходящая женщина , которая может стать ему женой , обратное верно и для женщин . но если учесть , что у человека может быть максимум несколько сотен знакомых , из которых лишь дюжина , а то и меньше , тех , кого он знает близко , а из этой дюжины у него один или от силы два друга , то можно легко увидеть , что с уч том миллионов живущих на земле людей , ни один подходящи

In [91]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [92]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [93]:
# Try experimenting with the size of that dataset
num_examples = 400000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [94]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = \
    train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

320000 320000 80000 80000


In [95]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [96]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
5 ----> я
65 ----> уже
6 ----> не
762 ----> занята
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
25 ----> i'm
39 ----> not
226 ----> busy
246 ----> anymore
3 ----> .
2 ----> <end>


In [119]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 512 #64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 512 #300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [120]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([512, 21]), TensorShape([512, 17]))

In [121]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=False,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [122]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (512, 1024)


In [123]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        # self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden):
        # enc_output shape == (batch_size, max_length, hidden_size)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x, initial_state=hidden)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [124]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)

In [125]:
decoder_sample_h.shape

TensorShape([512, 1024])

In [126]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [127]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [128]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [129]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.4998
Epoch 1 Batch 100 Loss 1.9838
Epoch 1 Batch 200 Loss 1.7446
Epoch 1 Batch 300 Loss 1.6382
Epoch 1 Batch 400 Loss 1.5669
Epoch 1 Batch 500 Loss 1.5522
Epoch 1 Batch 600 Loss 1.3938
Epoch 1 Loss 1.7202
Time taken for 1 epoch 200.65352416038513 sec

Epoch 2 Batch 0 Loss 1.3411
Epoch 2 Batch 100 Loss 1.1713
Epoch 2 Batch 200 Loss 1.0809
Epoch 2 Batch 300 Loss 0.9464
Epoch 2 Batch 400 Loss 0.8598
Epoch 2 Batch 500 Loss 0.7001
Epoch 2 Batch 600 Loss 0.7043
Epoch 2 Loss 0.9664
Time taken for 1 epoch 186.5998249053955 sec

Epoch 3 Batch 0 Loss 0.5863
Epoch 3 Batch 100 Loss 0.5415
Epoch 3 Batch 200 Loss 0.4690
Epoch 3 Batch 300 Loss 0.5108
Epoch 3 Batch 400 Loss 0.4495
Epoch 3 Batch 500 Loss 0.4295
Epoch 3 Batch 600 Loss 0.4037
Epoch 3 Loss 0.4792
Time taken for 1 epoch 188.19484949111938 sec

Epoch 4 Batch 0 Loss 0.3163
Epoch 4 Batch 100 Loss 0.2702
Epoch 4 Batch 200 Loss 0.3003
Epoch 4 Batch 300 Loss 0.3057
Epoch 4 Batch 400 Loss 0.3011
Epoch 4 Batch 500 Loss 0.267

In [130]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden)

        # storing the attention weights to plot later on
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [131]:
def translate(sentence):
    result, sentence = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [132]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

##### Перевод при изначальном запуске

In [ ]:
#Input: <start> здесь хорошо . <end>
#Predicted translation: it is good weather . <end> 
#Input: <start> я не смогу поехать . <end>
#Predicted translation: i can't go . <end> 
#Input: <start> вы еще дома ? <end>
#Predicted translation: are you still home ? <end> 
#Input: <start> вы все еще дома ? <end>
#Predicted translation: are you still at home ? <end> 
#Input: <start> попробуй сделать это . <end>
#Predicted translation: try to do it . <end> 
#Input: <start> я люблю , когда идет снег . <end>
#Predicted translation: i like what i seem . <end> 
#Input: <start> я никогда такого не делаю . <end>
#Predicted translation: i never do that . <end> 

##### Перевод после улучшения модели

In [133]:
translate('Здесь хорошо.')
translate('Я не смогу поехать.')
translate(u'Вы еще дома?')
translate(u'Вы все еще дома?')
translate(u'Попробуй сделать это.')
translate(u'Я люблю, когда идет снег.')
translate(u'Я никогда такого не делаю.')

Input: <start> здесь хорошо . <end>
Predicted translation: this is good here . <end> 
Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 
Input: <start> вы еще дома ? <end>
Predicted translation: are you still home ? <end> 
Input: <start> вы все еще дома ? <end>
Predicted translation: are you still at home ? <end> 
Input: <start> попробуй сделать это . <end>
Predicted translation: try to do it . <end> 
Input: <start> я люблю , когда идет снег . <end>
Predicted translation: i like it when it snows . <end> 
Input: <start> я никогда такого не делаю . <end>
Predicted translation: i never do that . <end> 


##### Выводы:
Увеличил размер обучаемого датасета и размер эмбединга. В результате явно неправильные переводы улучшились.<br>
Например:<br>
для "здесь хорошо" было "it is good weather", а стало "this is good here",<br>
для "я люблю , когда идет снег" было "i like what i seem", а стало "i like it when it snows".<br>